In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

#Fecha
import locale
locale.setlocale(locale.LC_ALL,'es_ES.UTF-8')

import dateparser
import datetime as datet
from datetime import datetime
sns.set()
date_fmt = '%b %Y'

#Graficación
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
comments_genre_df = pd.read_csv('./datasets/cwg.csv')

print(len(comments_genre_df))
comments_genre_df.head()

771502


,post_time,post_id,text,user,user_comment,text_proc,user_comment_proc,user_name_1,user_name_2,user_name_3,...,SEXO,likes,comments,shares,user_id,username,reactions,categorie,profile_owner,_merge
0,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Luz Marina Cuevas Valderrama,"Que triste noticia, lamentablemente su falleci...",hoy lamento profundamente el fallecimiento del...,que triste noticia lamentablemente su fallecim...,Luz,Marina,Cuevas,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
1,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Ana Josef Aparicio,Descansa en paz mi doctor de los leticianos ve...,hoy lamento profundamente el fallecimiento del...,descansa en paz mi doctor de los leticianos ve...,Ana,Josef,Aparicio,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
2,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Jorge Enrique Peña,Descanse en Paz gran PROFESIONAL ...,hoy lamento profundamente el fallecimiento del...,descanse en paz gran profesional ...,Jorge,Enrique,Peña,...,m,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
3,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Yolanda Murayari,"Qué mi Dios lo tenga en su Santo Reino,paz en ...",hoy lamento profundamente el fallecimiento del...,que mi dios lo tenga en su santo reinopaz en s...,Yolanda,Murayari,no_data,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
4,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Maruja Quiñonez,Hay luto y trztesa en mi corazon con la partid...,hoy lamento profundamente el fallecimiento del...,hay luto y trztesa en mi corazon con la partid...,Maruja,Quiñonez,no_data,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both


# 1. Cantidad de comentarios por día - histórico

In [3]:
df_tot_by_day = comments_genre_df.groupby(['post_time']).size()
df_tot_by_day = df_tot_by_day.to_frame()
df_tot_by_day.reset_index(inplace=True)

df_tot_by_day[['post_date','post_time']] = df_tot_by_day['post_time'].str.split(' ',expand=True)
df_tot_by_day = df_tot_by_day.rename(columns={0: "count"})
df_tot_by_day = df_tot_by_day[['post_date','post_time', 'count']]

df_tot_by_day

,post_date,post_time,count
0,2013-12-27,16:00:20,5
1,2013-12-30,16:01:21,9
2,2014-01-07,18:39:00,1
3,2014-01-11,07:46:53,1
4,2014-01-12,10:37:44,14
...,...,...,...
10018,2021-10-05,11:25:49,6
10019,2021-10-05,16:30:01,47
10020,2021-10-06,11:51:09,27
10021,2021-10-06,17:45:04,20


In [4]:
h_comment_vals = list(df_tot_by_day['post_time'])

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=list(df_tot_by_day['post_date']), y=h_comment_vals,
                    mode='markers',
                    marker=dict(size=3),
                    name='Datetime inicio'))

fig1.update_layout(title='2013 - 2021: Distribución diaria de comentarios',
                  legend_title="Time marks",
                  xaxis_tickangle=-45,
                  yaxis=dict(
                      title='Hora',
                      titlefont_size=13,
                      tickfont_size=12
                  ),
                  xaxis=dict(
                      title='Fecha',
                      titlefont_size=13,
                      tickfont_size=12
                  ),
                  autosize=False,
                  width=950,
                  height=400,
                  margin=dict(l=0, r=0, t=50, b=0),
                  font=dict(size=10))

fig1.update_yaxes(categoryorder='category ascending')

fig1.show()

# 1.1. Cantidad de comentarios por día de la semana

In [5]:
df_tot_by_day_count = df_tot_by_day.groupby(['post_date'])['count'].sum().to_frame()
df_tot_by_day_count.reset_index(inplace=True)
df_tot_by_day_count['post_date'] = pd.to_datetime(df_tot_by_day_count['post_date'], errors='coerce')
df_tot_by_day_count['post_date'] = df_tot_by_day_count['post_date'].dt.normalize()
df_tot_by_day_count['day_of_week'] = df_tot_by_day_count['post_date'].dt.day_name()

df_tot_by_day_count = df_tot_by_day_count[['post_date','day_of_week', 'count']]
df_tot_by_day_count = df_tot_by_day_count.groupby(['day_of_week'])['count'].sum().to_frame()
df_tot_by_day_count.reset_index(inplace=True)

di = {"Monday":"Lunes", "Tuesday":"Martes", "Wednesday":"Miércoles", "Thursday":"Jueves","Friday":"Viernes",
      "Saturday":"Sábado", "Sunday":"Domingo"}

df_tot_by_day_count = df_tot_by_day_count.replace({"day_of_week": di})

df_tot_by_day_count

,day_of_week,count
0,Viernes,106868
1,Lunes,103679
2,Sábado,94060
3,Domingo,96605
4,Jueves,127451
5,Martes,118531
6,Miércoles,124308


In [6]:
labels = df_tot_by_day_count['day_of_week'].value_counts().index
values = df_tot_by_day_count['count'].values

fig2 = make_subplots(rows=1, cols=3, specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])
fig2.add_trace(go.Pie(labels=labels, values=values), row=1, col=2)

# Use `hole` to create a donut-like pie chart
fig2.update_traces(hole=.4, hoverinfo="label+percent")

fig2.update_layout(
    title_text="2013 - 2021: % Total de comentarios por día de la semana",
    legend_title="Día",
    # Add annotations in the center of the donut pies.
    autosize=False,
    width=950,
    height=400,
    margin=dict(l=0, r=0, t=50, b=0),
    font=dict(size=10))

fig2.show()

# 2. Género

In [7]:
comments_genre_df.head()

,post_time,post_id,text,user,user_comment,text_proc,user_comment_proc,user_name_1,user_name_2,user_name_3,...,SEXO,likes,comments,shares,user_id,username,reactions,categorie,profile_owner,_merge
0,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Luz Marina Cuevas Valderrama,"Que triste noticia, lamentablemente su falleci...",hoy lamento profundamente el fallecimiento del...,que triste noticia lamentablemente su fallecim...,Luz,Marina,Cuevas,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
1,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Ana Josef Aparicio,Descansa en paz mi doctor de los leticianos ve...,hoy lamento profundamente el fallecimiento del...,descansa en paz mi doctor de los leticianos ve...,Ana,Josef,Aparicio,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
2,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Jorge Enrique Peña,Descanse en Paz gran PROFESIONAL ...,hoy lamento profundamente el fallecimiento del...,descanse en paz gran profesional ...,Jorge,Enrique,Peña,...,m,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
3,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Yolanda Murayari,"Qué mi Dios lo tenga en su Santo Reino,paz en ...",hoy lamento profundamente el fallecimiento del...,que mi dios lo tenga en su santo reinopaz en s...,Yolanda,Murayari,no_data,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
4,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Maruja Quiñonez,Hay luto y trztesa en mi corazon con la partid...,hoy lamento profundamente el fallecimiento del...,hay luto y trztesa en mi corazon con la partid...,Maruja,Quiñonez,no_data,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both


# Stacked bar - distro género por año

In [8]:
gen_month_tot = comments_genre_df[['post_time', 'user', 'SEXO']]
print(len(gen_month_tot))

gen_month_tot = gen_month_tot.drop_duplicates(subset='user', keep="first")
print(len(gen_month_tot))

gen_month_tot[['post_date','post_time']] = gen_month_tot['post_time'].str.split(' ',expand=True)
gen_month_tot[['post_date_year','post_date_month','post_date_day']] = gen_month_tot['post_date'].str.split('-',expand=True)
gen_month_tot = gen_month_tot[['post_date_year','post_date_month', 'SEXO']]
gen_month_tot_g = gen_month_tot.groupby(['post_date_year', 'post_date_month', 'SEXO'])['SEXO'].size().to_frame()
gen_month_tot_g = gen_month_tot_g.rename(columns={'SEXO': "count_genre"})

gen_month_tot_g = gen_month_tot_g.reset_index()
gen_month_tot_g

771502
265987


,post_date_year,post_date_month,SEXO,count_genre
0,2013,12,f,1
1,2013,12,m,8
2,2014,01,ND3,2
3,2014,01,f,12
4,2014,01,m,35
...,...,...,...,...
285,2021,09,m,3237
286,2021,10,ND2,1
287,2021,10,ND3,23
288,2021,10,f,716


In [9]:
fig3 = px.bar(gen_month_tot_g, x="post_date_year", y="count_genre", color="SEXO", barmode='stack')
fig3.update_layout(legend_title="Género", xaxis_title='Año', yaxis_title='Total de usuarios')

fig3.update_layout(title='2013 - 2021: Distribución anual de género de comentadores',
                  legend_title="Género",
                  autosize=False,
                  width=950,
                  height=400,
                  margin=dict(l=0, r=0, t=50, b=0),
                  font=dict(size=10))

fig3.show()

In [10]:
comments_genre_df.columns

Index(['post_time', 'post_id', 'text', 'user', 'user_comment', 'text_proc',
       'user_comment_proc', 'user_name_1', 'user_name_2', 'user_name_3',
       'user_name_4', 'New_ID', 'SEXO', 'likes', 'comments', 'shares',
       'user_id', 'username', 'reactions', 'categorie', 'profile_owner',
       '_merge'],
      dtype='object')

In [11]:
comments_genre_df.head()

,post_time,post_id,text,user,user_comment,text_proc,user_comment_proc,user_name_1,user_name_2,user_name_3,...,SEXO,likes,comments,shares,user_id,username,reactions,categorie,profile_owner,_merge
0,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Luz Marina Cuevas Valderrama,"Que triste noticia, lamentablemente su falleci...",hoy lamento profundamente el fallecimiento del...,que triste noticia lamentablemente su fallecim...,Luz,Marina,Cuevas,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
1,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Ana Josef Aparicio,Descansa en paz mi doctor de los leticianos ve...,hoy lamento profundamente el fallecimiento del...,descansa en paz mi doctor de los leticianos ve...,Ana,Josef,Aparicio,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
2,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Jorge Enrique Peña,Descanse en Paz gran PROFESIONAL ...,hoy lamento profundamente el fallecimiento del...,descanse en paz gran profesional ...,Jorge,Enrique,Peña,...,m,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
3,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Yolanda Murayari,"Qué mi Dios lo tenga en su Santo Reino,paz en ...",hoy lamento profundamente el fallecimiento del...,que mi dios lo tenga en su santo reinopaz en s...,Yolanda,Murayari,no_data,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both
4,2021-03-22 17:36:51,7.561767e+14,Hoy lamento profundamente el fallecimiento del...,Maruja Quiñonez,Hay luto y trztesa en mi corazon con la partid...,hoy lamento profundamente el fallecimiento del...,hay luto y trztesa en mi corazon con la partid...,Maruja,Quiñonez,no_data,...,f,26.0,11.0,0.0,3.221479e+14,Jorge Luis Mendoza,void_error,mayors,AlcaldeJorgeMendoza,both


In [12]:
cat_per_year = comments_genre_df[['post_time', 'post_id', 'text', 'user', 'categorie']]

cat_per_year[['post_date','post_time']] = cat_per_year['post_time'].str.split(' ',expand=True)
cat_per_year[['post_date_year','post_date_month','post_date_day']] = cat_per_year['post_date'].str.split('-',expand=True)
cat_per_year = cat_per_year.groupby(['post_date_year', 'post_date_month', 'categorie'])['categorie'].size().to_frame()
cat_per_year = cat_per_year.rename(columns={'categorie': "count_categorie"})

cat_per_year = cat_per_year.reset_index()
cat_per_year

C:\Users\user\anaconda3\envs\py38dane\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,post_date_year,post_date_month,categorie,count_categorie
0,2013,12,mayors,14
1,2014,01,mayors,85
2,2014,02,mayors,134
3,2014,03,mayors,214
4,2014,04,mayors,16
...,...,...,...,...
196,2021,07,politics,160
197,2021,07,sports,5059
198,2021,08,mayors,10484
199,2021,09,mayors,11899


In [13]:
fig4 = px.scatter(cat_per_year, x="post_date_year", y="post_date_month", color="categorie",
                 size='count_categorie')

fig4.update_layout(title='2013 - 2021: Distribución de posts por fecha y categoría',
                  legend_title="Categorías",
                  xaxis_tickangle=-45,
                  yaxis=dict(
                      title='Mes',
                      titlefont_size=13,
                      tickfont_size=12
                  ),
                  xaxis=dict(
                      title='Año',
                      titlefont_size=13,
                      tickfont_size=12
                  ),
                  autosize=False,
                  width=950,
                  height=400,
                  margin=dict(l=0, r=0, t=50, b=0),
                  font=dict(size=10))

fig4.update_yaxes(categoryorder='category ascending')


fig4.show()

In [14]:
def figures_to_html(figs, filename="dashboard_percepciondisc_dane.html"):
    dashboard = open(filename, 'w')
    dashboard.write("<html><head></head><body>" + "\n")
    #dashboard.write()
    for fig in figs:
        inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
        dashboard.write(inner_html)
    dashboard.write("</body></html>" + "\n")

In [15]:
figures_to_html([fig1, fig2, fig3, fig4])